In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from deepface import DeepFace
from google.colab import files

def is_black_frame(frame, threshold=15):
    """Check if frame is mostly black"""
    mean_brightness = np.mean(frame)
    return mean_brightness < threshold

def has_valid_face(frame, min_face_size=50):
    """Check if frame has a detectable face of reasonable size"""
    try:
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4, minSize=(min_face_size, min_face_size))
        return len(faces) > 0
    except:
        return False

def get_neutral_aus(frame_count, fps):
    """Return neutral AU values for frames without faces"""
    return {
        'frame': frame_count,
        'timestamp': frame_count / fps,
        'AU01': 0.0, 'AU02': 0.0, 'AU04': 0.0, 'AU05': 0.0, 'AU06': 0.0,
        'AU07': 0.0, 'AU09': 0.0, 'AU10': 0.0, 'AU12': 0.0, 'AU14': 0.0,
        'AU15': 0.0, 'AU17': 0.0, 'AU20': 0.0, 'AU23': 0.0, 'AU25': 0.0,
        'AU26': 0.0, 'AU28': 0.0,
        'face_detected': False,
        'is_black_frame': True
    }

def emotions_to_aus(emotions):
    """Convert emotion probabilities to Action Unit intensities"""
    # Normalize emotions to 0-1 range
    happy = emotions.get('happy', 0) / 100
    sad = emotions.get('sad', 0) / 100
    angry = emotions.get('angry', 0) / 100
    surprise = emotions.get('surprise', 0) / 100
    fear = emotions.get('fear', 0) / 100
    disgust = emotions.get('disgust', 0) / 100

    aus = {}
    # Map emotions to Action Units based on FACS
    aus['AU01'] = surprise * 0.8 + fear * 0.3
    aus['AU02'] = surprise * 0.9 + fear * 0.4
    aus['AU04'] = angry * 0.8 + disgust * 0.4
    aus['AU05'] = surprise * 0.7 + fear * 0.5
    aus['AU06'] = happy * 0.9
    aus['AU07'] = angry * 0.4 + disgust * 0.5
    aus['AU09'] = disgust * 0.8
    aus['AU10'] = disgust * 0.6 + angry * 0.3
    aus['AU12'] = happy * 1.0
    aus['AU14'] = happy * 0.6
    aus['AU15'] = sad * 0.8
    aus['AU17'] = sad * 0.5 + disgust * 0.3
    aus['AU20'] = fear * 0.6
    aus['AU23'] = angry * 0.5
    aus['AU25'] = surprise * 0.8 + fear * 0.4
    aus['AU26'] = surprise * 0.9 + fear * 0.5
    aus['AU28'] = fear * 0.3

    for au in aus:
        aus[au] = min(1.0, max(0.0, aus[au]))
    return aus

def process_video_to_aus(video_path):
    """
    Main function to process video path and download AU CSV.
    """
    # Check if file exists
    if not os.path.exists(video_path):
        print(f"❌ Error: File not found at {video_path}")
        return

    # Extract filename for saving later (e.g., "video.mov" -> "video.csv")
    base_name = os.path.basename(video_path)
    file_name_only = os.path.splitext(base_name)[0]
    csv_filename = f"{file_name_only}.csv"

    cap = cv2.VideoCapture(video_path)
    results = []
    frame_count = 0
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Safety check for FPS
    if fps == 0 or np.isnan(fps): 
        fps = 30.0
        
    print(f"🎬 Processing: {base_name}")
    print(f"📂 Output will be: {csv_filename}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Process every 30th frame (1 per second approx)
        if frame_count % 30 == 0:

            if is_black_frame(frame):
                frame_data = get_neutral_aus(frame_count, fps)
                results.append(frame_data)

            elif not has_valid_face(frame):
                frame_data = get_neutral_aus(frame_count, fps)
                frame_data['is_black_frame'] = False
                results.append(frame_data)

            else:
                try:
                    analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
                    if isinstance(analysis, list):
                        analysis = analysis[0]

                    aus = emotions_to_aus(analysis['emotion'])

                    frame_data = {
                        'frame': frame_count,
                        'timestamp': frame_count / fps,
                        'face_detected': True,
                        'is_black_frame': False
                    }
                    frame_data.update(aus)
                    results.append(frame_data)

                except Exception as e:
                    # Fallback for analysis errors
                    frame_data = get_neutral_aus(frame_count, fps)
                    frame_data['is_black_frame'] = False
                    results.append(frame_data)

        frame_count += 1
        if frame_count % 500 == 0:
            print(f"  ...processed {frame_count} frames")

    cap.release()

    # Create DataFrame
    df = pd.DataFrame(results)
    
    if len(df) == 0:
        print("No frames processed.")
        return

    # Prepare AU Columns
    au_columns = [col for col in df.columns if col.startswith('AU')]
    
    # Save CSV
    au_only_df = df[['timestamp'] + au_columns].copy()
    au_only_df.to_csv(csv_filename, index=False)
    
    print(f"\nProcessing Complete!")
    print(f"Data points saved: {len(df)}")
    print(f"⬇Downloading: {csv_filename}")
    
    files.download(csv_filename)

In [ ]:
process_video_to_aus("/content/drive/MyDrive/DatasetCercetare/VideosEdited/Britney_on_MarshallBritney.mov")